In [73]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

'''
training_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/train.csv',
index_col= [0,1],header=[0,1])
valid_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/valid.csv',
index_col= [0,1],header=[0,1])
test_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/test.csv',
index_col= [0,1],header=[0,1])
'''
df_data  = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/dataset.csv',
index_col= [0,1],header=[0,1])

In [74]:
df_data  =  df_data.dropna(subset=[('label', 'label')])

In [75]:
df_data =  df_data.dropna(axis=1,how='all')

In [76]:
df_data

feature                                          \
datetime                 KMID      KLEN     KMID2       KUP      KUP2   
2018/7/2   SH600096 -0.023715  0.039526 -0.600002  0.007905  0.200001   
           SZ000538 -0.012980  0.039593 -0.327829  0.019610  0.495283   
           SZ000560 -0.013699  0.032534 -0.421052  0.006849  0.210527   
           SZ000878 -0.031414  0.041885 -0.750000  0.001047  0.025000   
           SZ002265 -0.013841  0.040657 -0.340427  0.009516  0.234042   
...                       ...       ...       ...       ...       ...   
2020/12/31 SZ002059  0.051111  0.068889  0.741936  0.013333  0.193548   
           SZ002265  0.013908  0.020862  0.666665  0.001391  0.066672   
           SZ002428 -0.000813  0.024390 -0.033333  0.013008  0.533333   
           SZ002727  0.015549  0.027134  0.573035  0.006402  0.235955   
           SZ002750  0.001195  0.031063  0.038462  0.017921  0.576925   

                                                                       ...  \
datetime                 KLOW     KLOW2      KSFT     KSFT2     OPEN0  ...   
2018/7/2   SH600096  0.007905  0.199998 -0.023716 -0.600004  1.024291  ...   
           SZ000538  0.007004  0.176888 -0.025586 -0.646225  1.013151  ...   
           SZ000560  0.011986  0.368421 -0.008562 -0.263158  1.013889  ...   
           SZ000878  0.009424  0.224999 -0.023037 -0.550002  1.032432  ...   
           SZ002265  0.017301  0.425531 -0.006055 -0.148938  1.014035  ...   
...                       ...       ...       ...       ...       ...  ...   
2020/12/31 SZ002059  0.004444  0.064516  0.042222  0.612904  0.951374  ...   
           SZ002265  0.005563  0.266663  0.018080  0.866657  0.986283  ...   
           SZ002428  0.010569  0.433335 -0.003252 -0.133331  1.000814  ...   
           SZ002727  0.005183  0.191011  0.014329  0.528091  0.984689  ...   
           SZ002750  0.011947  0.384613 -0.004779 -0.153849  0.998807  ...   

                                                                       \
datetime              VSUMN10   VSUMN20   VSUMN30   VSUMN60    VSUMD5   
2018/7/2   SH600096  0.461892  0.428413  0.565435  0.523474  0.260493   
           SZ000538  0.459554  0.559512  0.503266  0.513980  0.519845   
           SZ000560  0.460508  0.517204  0.503589  0.494572 -0.555919   
           SZ000878  0.571208  0.490474  0.524230  0.506665  0.338845   
           SZ002265  0.633144  0.526052  0.505745  0.538933  0.238399   
...                       ...       ...       ...       ...       ...   
2020/12/31 SZ002059  0.260275  0.360533  0.457388  0.490975  0.616149   
           SZ002265  0.436146  0.521297  0.528495  0.490503 -0.335681   
           SZ002428  0.740589  0.514803  0.525479  0.497528 -0.540883   
           SZ002727  0.510551  0.479973  0.474285  0.486276  0.081809   
           SZ002750  0.851947  0.623012  0.537685  0.490309 -0.554905   

                                                                label  
datetime              VSUMD10   VSUMD20   VSUMD30   VSUMD60     label  
2018/7/2   SH600096  0.076217  0.143174 -0.130870 -0.046949 -0.005871  
           SZ000538  0.080893 -0.119023 -0.006531 -0.027960 -0.022544  
           SZ000560  0.078983 -0.034407 -0.007178  0.010856 -0.021239  
           SZ000878 -0.142416  0.019052 -0.048460 -0.013330 -0.013874  
           SZ002265 -0.266288 -0.052104 -0.011490 -0.077865 -0.041916  
...                       ...       ...       ...       ...       ...  
2020/12/31 SZ002059  0.479450  0.278934  0.085225  0.018049 -0.002137  
           SZ002265  0.127707 -0.042595 -0.056990  0.018994  0.007936  
           SZ002428 -0.481178 -0.029605 -0.050957  0.004944 -0.003984  
           SZ002727 -0.021102  0.040054  0.051429  0.027448 -0.005917  
           SZ002750 -0.703893 -0.246024 -0.075370  0.019383  0.038372  

[5965 rows x 158 columns]

In [78]:
import torch
import torch.nn as nn
import torch.nn.functional	
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
"""
数据预处理

train_data = training_data#训练集
test_data = test_data#测试集
#将数据集转换为tensor，因为PyTorch模型是使用tensor进行训练的，并将训练数据转换为输入序列和相应的标签
train_data = torch.tensor(train_data.values).contiguous().view(-1)
train_data = train_data.to(torch.float32)
test_data = torch.tensor(test_data.values).contiguous().view(-1)
test_data= test_data.to(torch.float32)
"""
#划分测试集和训练集



'\n数据预处理\n\ntrain_data = training_data#训练集\ntest_data = test_data#测试集\n#将数据集转换为tensor，因为PyTorch模型是使用tensor进行训练的，并将训练数据转换为输入序列和相应的标签\ntrain_data = torch.tensor(train_data.values).contiguous().view(-1)\ntrain_data = train_data.to(torch.float32)\ntest_data = torch.tensor(test_data.values).contiguous().view(-1)\ntest_data= test_data.to(torch.float32)\n'

In [79]:
df = df_data['feature'].dropna(axis=1,how='any')


In [80]:
df

datetime             CNTP5  CNTP10  CNTP20    CNTP30    CNTP60  CNTN5  CNTN10  \
2018/7/2   SH600096    0.4     0.4    0.30  0.266667  0.400000    0.6     0.6   
           SZ000538    0.2     0.4    0.35  0.433333  0.500000    0.8     0.6   
           SZ000560    0.4     0.5    0.45  0.400000  0.433333    0.6     0.4   
           SZ000878    0.4     0.3    0.45  0.433333  0.466667    0.6     0.7   
           SZ002265    0.6     0.6    0.55  0.566667  0.500000    0.4     0.4   
...                    ...     ...     ...       ...       ...    ...     ...   
2020/12/31 SZ002059    0.6     0.3    0.25  0.266667  0.366667    0.2     0.6   
           SZ002265    0.6     0.3    0.45  0.400000  0.466667    0.4     0.7   
           SZ002428    0.6     0.5    0.45  0.500000  0.550000    0.4     0.4   
           SZ002727    0.8     0.5    0.45  0.533333  0.500000    0.2     0.5   
           SZ002750    0.4     0.3    0.30  0.366667  0.400000    0.4     0.6   

datetime             CNTN20    CNTN30    CNTN60  CNTD5  CNTD10  CNTD20  \
2018/7/2   SH600096    0.65  0.666667  0.550000   -0.2    -0.2   -0.35   
           SZ000538    0.65  0.566667  0.483333   -0.6    -0.2   -0.30   
           SZ000560    0.50  0.566667  0.550000   -0.2     0.1   -0.05   
           SZ000878    0.55  0.533333  0.516667   -0.2    -0.4   -0.10   
           SZ002265    0.45  0.433333  0.500000    0.2     0.2    0.10   
...                     ...       ...       ...    ...     ...     ...   
2020/12/31 SZ002059    0.70  0.666667  0.550000    0.4    -0.3   -0.45   
           SZ002265    0.55  0.566667  0.500000    0.2    -0.4   -0.10   
           SZ002428    0.45  0.433333  0.416667    0.2     0.1    0.00   
           SZ002727    0.55  0.466667  0.500000    0.6     0.0   -0.10   
           SZ002750    0.65  0.566667  0.533333    0.0    -0.3   -0.35   

datetime               CNTD30    CNTD60  
2018/7/2   SH600096 -0.400000 -0.150000  
           SZ000538 -0.133333  0.016667  
           SZ000560 -0.166667 -0.116667  
           SZ000878 -0.100000 -0.050000  
           SZ002265  0.133333  0.000000  
...                       ...       ...  
2020/12/31 SZ002059 -0.400000 -0.183333  
           SZ002265 -0.166667 -0.033333  
           SZ002428  0.066667  0.133333  
           SZ002727  0.066667  0.000000  
           SZ002750 -0.200000 -0.133333  

[5965 rows x 15 columns]

In [81]:
df = df.dropna(axis=1,how='any')

In [82]:
df_data['label']

datetime                label
2018/7/2   SH600096 -0.005871
           SZ000538 -0.022544
           SZ000560 -0.021239
           SZ000878 -0.013874
           SZ002265 -0.041916
...                       ...
2020/12/31 SZ002059 -0.002137
           SZ002265  0.007936
           SZ002428 -0.003984
           SZ002727 -0.005917
           SZ002750  0.038372

[5965 rows x 1 columns]

In [84]:

# 创建两个列表，用来存储数据的特征和标签
data_feat, data_target = [],[]

# 设每条数据序列有20组数据
seq = 10
feature_num = 10

for index in range(len(df_data) - seq):
    # 构建特征集
    data_feat.append(df.iloc[:,range(feature_num)][index: index + seq].values)
    # 构建target集
    data_target.append(df_data['label'][index:index + seq])

# 将特征集和标签集整理成numpy数组
data_feat = np.array(data_feat)
data_target = np.array(data_target)


In [85]:
# 这里按照8:2的比例划分训练集和测试集
test_set_size = int(np.round(0.1*df_data.shape[0]))  # np.round(1)是四舍五入，
train_size = data_feat.shape[0] - (test_set_size) 
print(test_set_size)  # 输出测试集大小
print(train_size)     # 输出训练集大小

596
5359


In [86]:

trainX = torch.from_numpy(data_feat[:train_size].reshape(-1,seq,feature_num)).type(torch.Tensor)   
# 这里第一个维度自动确定，我们认为其为batch_size，因为在LSTM类的定义中，设置了batch_first=True
testX  = torch.from_numpy(data_feat[train_size:].reshape(-1,seq,feature_num)).type(torch.Tensor)
trainY = torch.from_numpy(data_target[:train_size].reshape(-1,seq,1)).type(torch.Tensor)
testY  = torch.from_numpy(data_target[train_size:].reshape(-1,seq,1)).type(torch.Tensor)
print('x_train.shape = ',trainX.shape)
print('y_train.shape = ',trainY.shape)
print('x_test.shape = ',testX.shape)
print('y_test.shape = ',testY.shape)



x_train.shape =  torch.Size([5359, 10, 10])
y_train.shape =  torch.Size([5359, 10, 1])
x_test.shape =  torch.Size([596, 10, 10])
y_test.shape =  torch.Size([596, 10, 1])


In [87]:
batch_size=5
train = torch.utils.data.TensorDataset(trainX,trainY)
test = torch.utils.data.TensorDataset(testX,testY)
train_loader = torch.utils.data.DataLoader(dataset=train, 
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [88]:
import torch.nn as nn

input_dim = feature_num    # 数据的特征数
hidden_dim = 2    # 隐藏层的神经元个数
num_layers = 2     # LSTM的层数
output_dim = 1     # 预测值的特征数
                   #（这是预测股票价格，所以这里特征数是1，如果预测一个单词，那么这里是one-hot向量的编码长度）
class LSTM(nn.Module):
    """
        使用LSTM进行回归
        参数：
        - input_size: feature size
        - hidden_size: number of hidden units
        - output_size: number of output
        - num_layers: layers of LSTM to stack
    """
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, _x):
        x, _ = self.lstm(_x)  # _x is input, size (seq_len, batch, input_size)
        s, b, h = x.shape  # x is output, size (seq_len, batch, hidden_size)
        x = x.view(s * b, h)
        x = self.fc(x)
        x = x.view(s, b, -1)  # 把形状改回来
        return x

In [89]:
# 定义模型
num_epochs = 50
model = LSTM(input_size=input_dim, hidden_size=hidden_dim, output_size=output_dim, num_layers=num_layers)
print(model)
# 打印模型各层的参数尺寸
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())
    
loss_function = nn.MSELoss()  # 损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # 优化器
for epoch in range(num_epochs):
    out = model(trainX)
    loss = loss_function(out, trainY)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch + 1) % 2 == 0:
        print('Epoch: {}, Loss:{:.10f}'.format(epoch + 1, loss.item()))

LSTM(
  (lstm): LSTM(10, 2, num_layers=2)
  (fc): Linear(in_features=2, out_features=1, bias=True)
)
torch.Size([8, 10])
torch.Size([8, 2])
torch.Size([8])
torch.Size([8])
torch.Size([8, 2])
torch.Size([8, 2])
torch.Size([8])
torch.Size([8])
torch.Size([1, 2])
torch.Size([1])
Epoch: 2, Loss:0.3472167253
Epoch: 4, Loss:0.1392121613
Epoch: 6, Loss:0.0068360115
Epoch: 8, Loss:0.0467005856
Epoch: 10, Loss:0.0237676203
Epoch: 12, Loss:0.0109624630
Epoch: 14, Loss:0.0075723617
Epoch: 16, Loss:0.0056338222
Epoch: 18, Loss:0.0036567245
Epoch: 20, Loss:0.0019276178
Epoch: 22, Loss:0.0009477836
Epoch: 24, Loss:0.0008291744
Epoch: 26, Loss:0.0012399179
Epoch: 28, Loss:0.0016756260
Epoch: 30, Loss:0.0017863913
Epoch: 32, Loss:0.0015380586
Epoch: 34, Loss:0.0011426283
Epoch: 36, Loss:0.0008541172
Epoch: 38, Loss:0.0007909968
Epoch: 40, Loss:0.0008936894
Epoch: 42, Loss:0.0010129070
Epoch: 44, Loss:0.0010368943
Epoch: 46, Loss:0.0009589590
Epoch: 48, Loss:0.0008518136
Epoch: 50, Loss:0.0007909238
Ep

In [101]:
testX


tensor([[[0.4000, 0.3000, 0.3500,  ..., 0.6000, 0.5667, 0.5000],
         [0.2000, 0.2000, 0.3500,  ..., 0.6500, 0.6333, 0.5500],
         [0.4000, 0.4000, 0.3500,  ..., 0.6000, 0.5333, 0.5000],
         ...,
         [0.6000, 0.5000, 0.4000,  ..., 0.6000, 0.4667, 0.5000],
         [0.6000, 0.4000, 0.4000,  ..., 0.5500, 0.5333, 0.5833],
         [0.6000, 0.5000, 0.4000,  ..., 0.6000, 0.6000, 0.5667]],

        [[0.2000, 0.2000, 0.3500,  ..., 0.6500, 0.6333, 0.5500],
         [0.4000, 0.4000, 0.3500,  ..., 0.6000, 0.5333, 0.5000],
         [0.2000, 0.4000, 0.4500,  ..., 0.5500, 0.5333, 0.5000],
         ...,
         [0.6000, 0.4000, 0.4000,  ..., 0.5500, 0.5333, 0.5833],
         [0.6000, 0.5000, 0.4000,  ..., 0.6000, 0.6000, 0.5667],
         [0.6000, 0.4000, 0.4000,  ..., 0.5500, 0.5333, 0.5000]],

        [[0.4000, 0.4000, 0.3500,  ..., 0.6000, 0.5333, 0.5000],
         [0.2000, 0.4000, 0.4500,  ..., 0.5500, 0.5333, 0.5000],
         [0.2000, 0.4000, 0.4000,  ..., 0.5500, 0.4333, 0.

In [96]:
model = model.eval()  # 转换成测试模式
test_x = testX.reshape(-1,1, input_dim)
pred_y = model(testX)
pred_y = pred_y.view(-1).data.numpy()

In [105]:
loss = model(testX)-testY

In [106]:
loss = loss.view(-1).data.numpy()

In [107]:
loss

array([-0.02121804, -0.01663848, -0.02596271, ..., -0.00724346,
        0.00467736,  0.00661191], dtype=float32)